In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
import pandas as pd
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tqdm import tqdm

import pickle

from tensorflow.contrib.tensorboard.plugins import projector

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

## Step 1 - Get Data

In [3]:
zh_wiki_id = open("data/zh_wiki_id_w_d").readline()
word_to_id = pickle.load(open("data/word_to_id_w_d.pkl", "rb"))
id_to_word = pickle.load(open("data/id_to_word_w_d.pkl", "rb"))

# zh_wiki_id = open("data/zh_wiki_id").readline()
# word_to_id = pickle.load(open("data/word_to_id.pkl", "rb"))
# id_to_word = pickle.load(open("data/id_to_word.pkl", "rb"))
# word_count = pickle.load(open("data/count.pkl", "rb"))

In [4]:
len(id_to_word), len(word_to_id), len(zh_wiki_id)

(507260, 507260, 862838467)

In [5]:
def getWord(data, num, data_index):
    sub_data_string = data[data_index:data_index+num*(6+1)]
    result = []
    for index, item in enumerate(sub_data_string.split()):
        if index == num: break
        data_index += len(item) + 1
        result.append(int(item))
    if len(result) < num:
        return getWord(data, num, 0)
    assert len(result) == num
    return result, data_index

In [6]:
def generate_batch(batch_size, skip_window, num_skips):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size, num_skips), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    assert batch_size >= span
    buffer = collections.deque(maxlen=span)  # pylint: disable=redefined-builti
    
    result, data_index = getWord(zh_wiki_id, span, data_index)
    buffer.extend(result)
    
    for i in range(batch_size):
        context_words = [w for w in range(span) if w != skip_window]
        batch[i, :] = [buffer[token] for idx, token in enumerate(context_words)]
        labels[i, 0] = buffer[skip_window]
        result, data_index = getWord(zh_wiki_id, 1, data_index)
        buffer.append(result[0])
        if data_index > len(zh_wiki_id):
            result, data_index = getWord(zh_wiki_id, span-1, 0)
            buffer.extend(result)
        if i == batch_size - span:
            last_index = data_index
            
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = last_index
    return batch, labels

In [8]:
data_index = 0

batch, labels = generate_batch(batch_size=8, skip_window=1, num_skips=2*1)
for i in range(8):
    print(batch[i, 0], id_to_word[batch[i, 0]],
          batch[i, 1], id_to_word[batch[i, 1]],
          '->', labels[i, 0], id_to_word[labels[i, 0]])
data_index

1348 数学 501 利用 -> 9 是
9 是 237319 符号语言 -> 501 利用
501 利用 141 研究 -> 237319 符号语言
237319 符号语言 894 数量 -> 141 研究
141 研究 5 、 -> 894 数量
894 数量 499 结构 -> 5 、
5 、 5 、 -> 499 结构
499 结构 331 变化 -> 5 、


34

## Step 2 - Build & Train Network

In [ ]:
def _compute_sampled_logits(weights,
                            biases,
                            labels,
                            inputs,
                            num_sampled,
                            num_classes,
                            num_true=1,
#                             sampled_values=None,
                            subtract_log_q=True,
#                             remove_accidental_hits=False,
                            partition_strategy="mod",
                            name=None,
                            seed=None):

    # Sample the negative labels.
    #   sampled shape: [num_sampled] tensor
    #   true_expected_count shape = [batch_size, 1] tensor
    #   sampled_expected_count shape = [num_sampled] tensor
    sampled_values = candidate_sampling_ops.log_uniform_candidate_sampler(
          true_classes=labels,
          num_true=num_true,
          num_sampled=num_sampled,
          unique=True,
          range_max=num_classes,
          seed=seed)
    # NOTE: pylint cannot tell that 'sampled_values' is a sequence
    # pylint: disable=unpacking-non-sequence
    sampled, true_expected_count, sampled_expected_count = (
        array_ops.stop_gradient(s) for s in sampled_values)
    # pylint: enable=unpacking-non-sequence
    sampled = math_ops.cast(sampled, dtypes.int64)

    labels_flat = array_ops.reshape(labels, [-1])
    # labels_flat is a [batch_size * num_true] tensor
    # sampled is a [num_sampled] int tensor
    all_ids = array_ops.concat([labels_flat, sampled], 0)

    # Retrieve the true weights and the logits of the sampled weights.

    ########
    # all w
    ########
    # weights shape is [num_classes, dim]
    all_w = embedding_ops.embedding_lookup(
        weights, all_ids, partition_strategy=partition_strategy)
    # true_w shape is [batch_size * num_true, dim]
    true_w = array_ops.slice(
        all_w, [0, 0], array_ops.stack([array_ops.shape(labels_flat)[0], -1]))
    sampled_w = array_ops.slice(
        all_w, array_ops.stack([array_ops.shape(labels_flat)[0], 0]), [-1, -1]) 
    # inputs has shape [batch_size, dim]
    # sampled_w has shape [num_sampled, dim]
    # Apply X*W', which yields [batch_size, num_sampled]
    sampled_logits = math_ops.matmul(inputs, sampled_w, transpose_b=True)

    ########
    # all b 
    ########
    # Retrieve the true and sampled biases, compute the true logits, and
    # add the biases to the true and sampled logits.
    all_b = embedding_ops.embedding_lookup(
        biases, all_ids, partition_strategy=partition_strategy)
    # true_b is a [batch_size * num_true] tensor
    # sampled_b is a [num_sampled] float tensor
    true_b = array_ops.slice(all_b, [0], array_ops.shape(labels_flat))
    sampled_b = array_ops.slice(all_b, array_ops.shape(labels_flat), [-1])

    # inputs shape is [batch_size, dim]
    # true_w shape is [batch_size * num_true, dim]
    # row_wise_dots is [batch_size, num_true, dim]
    dim = array_ops.shape(true_w)[1:2]
    new_true_w_shape = array_ops.concat([[-1, num_true], dim], 0)
    row_wise_dots = math_ops.multiply(
        array_ops.expand_dims(inputs, 1),
        array_ops.reshape(true_w, new_true_w_shape))
    # We want the row-wise dot plus biases which yields a
    # [batch_size, num_true] tensor of true_logits.
    dots_as_matrix = array_ops.reshape(row_wise_dots,
                                       array_ops.concat([[-1], dim], 0))
    true_logits = array_ops.reshape(_sum_rows(dots_as_matrix), [-1, num_true])
    true_b = array_ops.reshape(true_b, [-1, num_true])
    true_logits += true_b
    sampled_logits += sampled_b


    if subtract_log_q:
        # Subtract log of Q(l), prior probability that l appears in sampled.
        true_logits -= math_ops.log(true_expected_count)
        sampled_logits -= math_ops.log(sampled_expected_count)

    # Construct output logits and labels. The true labels/logits start at col 0.
    out_logits = array_ops.concat([true_logits, sampled_logits], 1)

    # true_logits is a float tensor, ones_like(true_logits) is a float
    # tensor of ones. We then divide by num_true to ensure the per-example
    # labels sum to 1.0, i.e. form a proper probability distribution.
    out_labels = array_ops.concat([
        array_ops.ones_like(true_logits) / num_true,
        array_ops.zeros_like(sampled_logits)
    ], 1)

    return out_logits, out_labels

In [ ]:
batch_size = 100 # batch
embedding_size = 100    # Dimension of the embedding vector.
skip_window = 5    # How many words to consider left and right.
num_skips = 2*skip_window    # How many times to reuse an input to generate a label.
num_sampled = 100    # Number of negative examples to sample.
# num_sampled = 128    # Number of negative examples to sample.
vocabulary_size = len(id_to_word)

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
# valid_size = 16    # Random set of words to evaluate similarity on.
# valid_window = 100    # Only pick dev samples in the head of the distribution.
# valid_examples = np.random.choice(valid_window, valid_size, replace=False)
# valid_examples = list(range(1, 10))
valid_examples = list(range(280, 291))
valid_size = len(valid_examples)

graph = tf.Graph()

with graph.as_default():

    # Input data.
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size, num_skips])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/gpu:0'):
    # Look up embeddings for inputs.
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
            # take mean of embeddings of context words for context embedding
            embed_context = tf.reduce_mean(embed, 1)

    with tf.device('/gpu:0'):
        # Construct the variables for the NCE loss
        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                    tf.truncated_normal(
                            [vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
                tf.nn.nce_loss(
                        weights=nce_weights,
                        biases=nce_biases,
                        labels=train_labels,
                        inputs=embed_context,
#                         labels=embed_context,
#                         inputs=train_labels,
                        num_sampled=num_sampled,
                        num_classes=vocabulary_size))
        logits, labels = _compute_sampled_logits(
              weights=weights,
              biases=biases,
              labels=labels,
              inputs=inputs,
              num_sampled=num_sampled,
              num_classes=num_classes,
              num_true=num_true,
              sampled_values=sampled_values,
              subtract_log_q=True,
              remove_accidental_hits=remove_accidental_hits,
              partition_strategy=partition_strategy,
              name=name)
        sampled_losses = sigmoid_cross_entropy_with_logits(
              labels=labels, logits=logits, name="sampled_losses")
        loss = _sum_rows(sampled_losses)

    # Add the loss value as a scalar to summary.
    tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    with tf.name_scope('optimizer'):
#         optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
        optimizer = tf.train.GradientDescentOptimizer(1).minimize(loss)
#         optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    print(valid_embeddings, normalized_embeddings, similarity)

    # Merge all summaries.
    merged = tf.summary.merge_all()

    # Add variable initializer.
    init = tf.global_variables_initializer()

    # Create a saver.
    saver = tf.train.Saver()

Tensor("embedding_lookup:0", shape=(11, 200), dtype=float32) Tensor("truediv:0", shape=(507260, 200), dtype=float32) Tensor("MatMul:0", shape=(11, 507260), dtype=float32)
---- top k same word
test_embeddings Tensor("embedding_lookup_1:0", dtype=float32)
normalized_embeddings Tensor("truediv:0", shape=(507260, 200), dtype=float32)
similarity_test: Tensor("MatMul_1:0", shape=(?, 507260), dtype=float32)
similarity_test_top_k_value: Tensor("TopKV2:0", shape=(?, 5), dtype=float32)
similarity_test_top_k_index: Tensor("TopKV2:1", shape=(?, 5), dtype=int32)
---- analogical reasoning
test_embeddings: Tensor("embedding_lookup_3:0", dtype=float32, device=/device:GPU:0)
test_embs: Tensor("split_1:0", dtype=float32) Tensor("split_1:1", dtype=float32)
test_embs: Tensor("split_1:2", dtype=float32) Tensor("split_1:3", dtype=float32)
test_result Tensor("Add:0", dtype=float32)
normalized_embeddings Tensor("truediv:0", shape=(507260, 200), dtype=float32)
analogical_similarity Tensor("Squeeze:0", dtype=fl

In [ ]:
num_steps = 20000001
# log_dir = "./log_002_baseline_cbow/"
log_dir = "./log_008_simple_cbow/"

tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth = True

with tf.Session(graph=graph, config=tfconfig) as session:
    # Open a writer to write summaries.
    writer = tf.summary.FileWriter(log_dir, session.graph)

    # We must initialize all variables before we use them.
    init.run()
#     saver = tf.train.import_meta_graph('./checkpoint_dir/MyModel-1000.meta')
#     saver.restore(session, tf.train.latest_checkpoint(log_dir))
    print('Initialized')

    average_loss = 0
    start_index = 0
    for step in xrange(start_index, num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, skip_window=skip_window, num_skips=num_skips)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # Define metadata variable.
        run_metadata = tf.RunMetadata()

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
        # Feed metadata variable to session for visualizing the graph in TensorBoard.
        _, summary, loss_val = session.run(
                [optimizer, merged, loss],
                feed_dict=feed_dict,
                run_metadata=run_metadata)
        average_loss += loss_val
        
        # Add returned summaries to writer in each step.
        writer.add_summary(summary, step)
        # Add metadata to visualize the graph for the last run.
        if step == (num_steps - 1):
            writer.add_run_metadata(run_metadata, 'step%d' % step, data_index)

        if step % 2000 == 0:
            if step > 0 and step != start_index:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = id_to_word[valid_examples[i]]
                top_k = 8    # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = id_to_word[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
                
            # Save the model for checkpoints.
            saver.save(session, os.path.join(log_dir, 'model.ckpt'), global_step=step)
        
        if step % 1000000 == 0 and step != start_index:
            word2vec = embeddings.eval()
            print(word2vec.shape, type(word2vec))
            np.save("result/008#simple_cbow_"+str(step), word2vec)
            
    final_embeddings = normalized_embeddings.eval()

    # Write corresponding labels for the embeddings.
    with open(log_dir + '/metadata.tsv', 'w') as f:
        for i in xrange(vocabulary_size):
            f.write(id_to_word[i] + '\n')

    # Save the model for checkpoints.
    saver.save(session, os.path.join(log_dir, 'model.ckpt'), global_step=step)

    # Create a configuration for visualizing embeddings with the labels in TensorBoard.
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    embedding_conf.metadata_path = os.path.join(log_dir, 'metadata.tsv')
    projector.visualize_embeddings(writer, config)

writer.close()

Initialized
Average loss at step  0 :  537.719726562
Nearest to 音乐: 霸级, 百货店, 刘家, 海邦, 哈米特, 左大培, 王云峰, 无数次,
Nearest to 地方: 纳溪区, 正屿, 多时, 什尼基, 中十, 椰仁, 孙晓山, 蛋蛋,
Nearest to 社会: 夫在, 美满婚姻, 公共广播, 安襄, 看云起, 金磬, 史威瑟, 罚球点,
Nearest to 服务: 芥花油, 嘀, 建立联系, 遭杨, 宝塔式, 六十日, 黑岗, 八匹,
Nearest to 均: 水手, 元认知, 笑岩, 贝考, 灶君, 打打, 甲乙丙丁, 未闻,
Nearest to 型: 梵谷, 大原丽子, 矸石, 之诉, 读书处, 原理图, 证在, 极贵,
Nearest to 学生: 明生, 彭锋, 吴炳, 盖顿, 网络游戏, 不见经传, 忠尚, 掩住,
Nearest to 今: 大学物理, 马吉尔, 五星红旗, 素莲, 预制板, 远取, 襄壮, 拉远,
Nearest to 受到: 基二, 几十家, 岑敖晖, 上单, 雪岭, 众所周知, 康姆, 鲬,
Nearest to 事件: 红砂岩, 绑, 旗之台, 集散中心, 书单, 华镇, 蒂帕帕, 柴炭,
Nearest to 经济: 角位, 胺类, 蝴蝶梦, 主攻手, 最低点, 根属, 祭及, 设此,
Average loss at step  2000 :  299.886164371
Average loss at step  4000 :  210.939039188
Average loss at step  6000 :  172.896751801
Average loss at step  8000 :  149.274800348
Average loss at step  10000 :  131.997478508
Nearest to 音乐: 该, 刘家, 霸级, 无数次, 上木, 喜龙, 拖雷, 冰缝,
Nearest to 地方: >, <, а, 以及, 到, 的, 数学, 可以,
Nearest to 社会: 真姬, 伊兹图, 克朗, 公山, 蒋百, 耶律宗, 翌春, 修昔底,
Nearest to 服务: 芥花油, 嘀, 建立联系

Nearest to 经济: 国家, 制度, 政治, 文化, 以前, 发展, 政府, 该,
Average loss at step  112000 :  10.4218673595
Average loss at step  114000 :  9.84882834339
Average loss at step  116000 :  9.68243611526
Average loss at step  118000 :  9.71459861076
Average loss at step  120000 :  9.32366540265
Nearest to 音乐: →, 掌, 景, 模型, 该, ○, 寺, ●,
Nearest to 地方: 按, 景, 事务, 负责, ह, ्, 呼吁, 帮助,
Nearest to 社会: 文化, 发展, 经济, 暗杀, 致力于, 句, 批评, 提升,
Nearest to 服务: 宣布, ○, 班禅, ●, 主, 已, 世达赖, 十二,
Nearest to 均: 世达赖, 景, ┃, ○, ●, 人士, 班禅, !,
Nearest to 型: ○, ●, 景, ┃, 世达赖, 韵, の, 掌,
Nearest to 学生: 斗争, ○, の, α, 下午, 令, ●, 世达赖,
Nearest to 今: 景, ○, 世达赖, ●, ┃, 、, 山, 场区,
Nearest to 受到: 韵, ā, ○, ्, ●, 景, 及其, 遭到,
Nearest to 事件: 世达赖, ○, 遂, ●, 景, 韵, 一带, 掌,
Nearest to 经济: 国家, 制度, 政治, 发展, 社会, 文化, 政府, 以前,
Average loss at step  122000 :  9.46721331179
Average loss at step  124000 :  9.03812331605
Average loss at step  126000 :  8.93297407234
Average loss at step  128000 :  8.75401068735
Average loss at step  130000 :  8.43851564515
Nearest to 音乐: →, 景, 掌, 

Nearest to 均: 世达赖, ┃, 景, ﻿, ○, ●, র, 水文,
Nearest to 型: ﻿, ○, ●, ┃, 世达赖, 景, 逻辑学, 水文,
Nearest to 学生: 学校, 教育, 大学, 斗争, 中学, 学院, 课程, 团体,
Nearest to 今: 景, 山, ○, ﻿, 镇, 、, 一带, 世达赖,
Nearest to 受到: 韵, 遭到, 对, 受, 依旧, ā, ﻿, 朱,
Nearest to 事件: 发生, 世达赖, 革命, 遂, 凡莉嘉, ○, 对抗, 导致,
Nearest to 经济: 制度, 发展, 社会, 产业, 政策, 政治, 工业, 生活,
Average loss at step  232000 :  5.36487189078
Average loss at step  234000 :  5.50968352842
Average loss at step  236000 :  5.40923278713
Average loss at step  238000 :  5.26875572014
Average loss at step  240000 :  5.40395901823
Nearest to 音乐: 景, 电影, 游戏, 掌, →, 全面, 当代, ○,
Nearest to 地方: 事务, 政府, 市, 负责, 依, 防卫, 遣, 地区,
Nearest to 社会: 文化, 经济, 发展, 批评, 政治, 理论, 暗杀, 提升,
Nearest to 服务: 提供, ﻿, ○, 管理, ●, 大型, 宣布, 有限公司,
Nearest to 均: 世达赖, ┃, 景, ﻿, ○, ●, র, 水文,
Nearest to 型: ﻿, ○, ●, ┃, 世达赖, 水文, 逻辑学, 景,
Nearest to 学生: 学校, 教育, 斗争, 大学, 中学, 学院, 课程, 团体,
Nearest to 今: 景, 山, ○, ﻿, 一带, 镇, 世达赖, ●,
Nearest to 受到: 遭到, 韵, 对, 受, 依旧, ﻿, ā, 朱,
Nearest to 事件: 发生, 世达赖, 革命, 遂, ○, 中共, 对抗, 凡莉嘉,
Nearest to 经济: 制度, 社会, 

Average loss at step  342000 :  4.94251296639
Average loss at step  344000 :  4.93758272362
Average loss at step  346000 :  4.94579759502
Average loss at step  348000 :  4.8707535069
Average loss at step  350000 :  4.92402132297
Nearest to 音乐: 电影, 艺术, 作品, 专辑, お, 制作, 游戏, 文学,
Nearest to 地方: 地区, 市, 事务, 政府, 依, 一带, 国, 区域,
Nearest to 社会: 文化, 政治, 经济, 批评, 运动, 制度, 生活, 发展,
Nearest to 服务: 提供, 设施, 企业, 用户, 大型, 设有, 有限公司, 推出,
Nearest to 均: 相关, ┃, 世达赖, 供, క, 除, 景, 水手,
Nearest to 型: ﻿, ┃, క, ○, ●, 逻辑学, 水文, 世达赖,
Nearest to 学生: 学校, 教育, 学习, 课程, 中学, 大学, 小学, 斗争,
Nearest to 今: 一带, 山, 镇, 府, 城, 郡, 景, ○,
Nearest to 受到: 受, 遭到, 对, 韵, 依旧, 造成, 因素, 严重,
Nearest to 事件: 发生, 革命, 严重, 世达赖, 遭到, 质疑, 遂, 导致,
Nearest to 经济: 产业, 社会, 政策, 制度, 农业, 发展, 工业, 实行,
Average loss at step  352000 :  4.94347911775
Average loss at step  354000 :  4.90085094273
Average loss at step  356000 :  4.87051771891
Average loss at step  358000 :  4.89046028459
Average loss at step  360000 :  4.90786530244
Nearest to 音乐: 电影, 艺术, 作品, 专辑, 制作, お, 个人, 游戏,


Nearest to 均: 第二批, 皆, 相关, ┃, 世达赖, 供, క, ﻿,
Nearest to 型: ﻿, ┃, 发动机, WDW, 第二批, WD, 制造, ○,
Nearest to 学生: 学校, 教育, 课程, 学习, 中学, 小学, 大学, 学院,
Nearest to 今: 一带, 镇, 山, 城, 府, 景, 郡, 河南,
Nearest to 受到: 受, 遭到, 对, 因素, 造成, 韵, 引起, 依旧,
Nearest to 事件: 发生, 革命, 遭到, 质疑, 严重, 引起, 行为, 关注,
Nearest to 经济: 产业, 农业, 社会, 政策, 工业, 制度, 发展, 市场,
Average loss at step  462000 :  4.83105643249
Average loss at step  464000 :  4.69540683341
Average loss at step  466000 :  4.8086070224
Average loss at step  468000 :  4.8932597791
Average loss at step  470000 :  4.85886407197
Nearest to 音乐: 艺术, 专辑, 电影, 作品, 创作, 制作, 电视, 演出,
Nearest to 地方: 地区, 市, 一带, 事务, 区, 区域, 管辖, 口,
Nearest to 社会: 政治, 文化, 批评, 经济, 运动, 政策, 生活, 制度,
Nearest to 服务: 提供, 设施, 用户, 设有, 网站, 推出, 免费, 有限公司,
Nearest to 均: 皆, 第二批, 相关, 供, ┃, 世达赖, క, 除,
Nearest to 型: ﻿, ┃, WDW, 第二批, ○, 发动机, 制造, ●,
Nearest to 学生: 学校, 教育, 课程, 学习, 中学, 小学, 大学, 学院,
Nearest to 今: 一带, 山, 镇, 城, 府, 景, ○, 山西,
Nearest to 受到: 受, 遭到, 对, 因素, 造成, 韵, 引起, 依旧,
Nearest to 事件: 发生, 革命, 遭到, 质疑, 严重, 引起, 行为, 关注,
Neare

...

Nearest to 经济: 产业, 政策, 农业, 社会, 工业, 市场, 价值, 工资,
Average loss at step  912000 :  4.65031277561
Average loss at step  914000 :  4.64706834733
Average loss at step  916000 :  4.56164303374
Average loss at step  918000 :  4.50506230795
Average loss at step  920000 :  4.52087540293
Nearest to 音乐: 乐团, 歌手, 表演, 演奏, 歌曲, 唱片, 专辑, 作曲家,
Nearest to 地方: 地区, 市, 区域, 一带, 事务, 机关, 政府, 当地,
Nearest to 社会: 精神, 文化, 政治, 教育, 经济, 政策, 致力于, 自我,
Nearest to 服务: 免费, 来往, 邨, 设施, 提供, 改善, 网站, 屯门,
Nearest to 均: 皆, 都, 亦, 仍, 只, 宴请, 才, 供,
Nearest to 型: 发动机, 机车, WDW, 装备, 制造, 引擎, 动力, -,
Nearest to 学生: 学校, 课程, 教师, 教育, 该校, 中学, 学习, 小学,
Nearest to 今: 一带, 河南, 山西, 镇, 府, 山东, 陕西, 元年,
Nearest to 受到: 受, 遭到, 引起, 对, 造成, 因素, 对于, 中立,
Nearest to 事件: 报导, 事故, 行为, 引起, 质疑, 行动, 严重, 遭到,
Nearest to 经济: 产业, 政策, 农业, 社会, 工业, 价值, 市场, 改革,
Average loss at step  922000 :  4.47414002687
Average loss at step  924000 :  4.57993997645
Average loss at step  926000 :  4.60017517734
Average loss at step  928000 :  4.54123067224
Average loss at step  930000 :  4.5

Average loss at step  1024000 :  4.52662038815
Average loss at step  1026000 :  4.514716802
Average loss at step  1028000 :  4.47432625246
Average loss at step  1030000 :  4.43405824047
Nearest to 音乐: 乐团, 歌手, 演奏, 表演, 作曲家, 歌曲, 唱片, 专辑,
Nearest to 地方: 地区, 市, 区域, 当地, 机关, 一带, 政府, 事务,
Nearest to 社会: 精神, 政治, 文化, 教育, 经济, 致力于, 政策, 自我,
Nearest to 服务: 免费, 来往, 邨, 路线, 往返, 屯门, 设施, 改善,
Nearest to 均: 皆, 都, 亦, 仍, 宴请, 则, 只, 才,
Nearest to 型: 机车, 发动机, WDW, 装备, -, 辆, 动力, 制造,
Nearest to 学生: 学校, 教师, 课程, 该校, 教育, 中学, 小学, 学习,
Nearest to 今: 一带, 河南, 山西, 镇, 府, 陕西, 今属, 山东,
Nearest to 受到: 受, 遭到, 引起, 对, 造成, 因素, 深受, 中立,
Nearest to 事件: 报导, 事故, 行为, 引起, 质疑, 媒体, 引发, 行动,
Nearest to 经济: 产业, 政策, 农业, 社会, 工业, 价值, 市场, 工资,
Average loss at step  1032000 :  3.81906050372
Average loss at step  1034000 :  4.45146284503
Average loss at step  1036000 :  4.51937931347
Average loss at step  1038000 :  4.55537513721
Average loss at step  1040000 :  4.69024526072
Nearest to 音乐: 乐团, 作曲家, 演奏, 歌手, 表演, 唱片, 歌曲, 创作,
Nearest to 地方: 地区, 市, 当地, 区域

Average loss at step  1138000 :  4.52015492141
Average loss at step  1140000 :  4.48135055447
Nearest to 音乐: 乐团, 演奏, 歌手, 作曲家, 唱片, 表演, 歌曲, 专辑,
Nearest to 地方: 地区, 市, 区域, 机关, 政府, 当地, 一带, 事务,
Nearest to 社会: 精神, 教育, 政治, 经济, 致力于, 文化, 政策, 自我,
Nearest to 服务: 免费, 来往, 邨, 往返, 路线, 业务, 网站, 屯门,
Nearest to 均: 皆, 都, 仍, 亦, 则, 宴请, 顾及, 只,
Nearest to 型: 机车, 发动机, WDW, 装备, -, 辆, 动力, 车体,
Nearest to 学生: 学校, 课程, 教师, 该校, 中学, 小学, 教育, 高中,
Nearest to 今: 今属, 镇, 山西, 河南, 二年, 一带, 元年, 黄河,
Nearest to 受到: 受, 遭到, 引起, 对, 造成, 因素, 深受, 中立,
Nearest to 事件: 事故, 报导, 行为, 行动, 引发, 调查, 引起, 严重,
Nearest to 经济: 产业, 政策, 农业, 社会, 工业, 价值, 资源, 改革,
Average loss at step  1142000 :  4.48268823123
Average loss at step  1144000 :  4.54000239336
Average loss at step  1146000 :  4.4800455761
Average loss at step  1148000 :  4.50906739575
Average loss at step  1150000 :  4.47897221112
Nearest to 音乐: 乐团, 演奏, 歌手, 作曲家, 唱片, 表演, 歌曲, 专辑,
Nearest to 地方: 地区, 市, 区域, 机关, 政府, 当地, 事务, 一带,
Nearest to 社会: 精神, 教育, 政治, 致力于, 经济, 自我, 文化, 政策,
Nearest to 服务: 免费, 来往, 路线

Nearest to 均: 皆, 都, 仍, 亦, 则, 宴请, 顾及, 一致,
Nearest to 型: 机车, 发动机, WDW, 装备, 车型, 辆, 车体, 引擎,
Nearest to 学生: 学校, 课程, 教师, 该校, 中学, 小学, 教育, 入学,
Nearest to 今: 今属, 河南, 山西, 镇, 一带, 黄河, 二年, 陕西,
Nearest to 受到: 受, 遭到, 引起, 深受, 对, 造成, 因素, 遭受,
Nearest to 事件: 事故, 行动, 报导, 行为, 革命, 媒体, 暴力, 调查,
Nearest to 经济: 产业, 政策, 农业, 社会, 价值, 工业, 工资, 金融,
Average loss at step  1252000 :  4.47074299181
Average loss at step  1254000 :  4.46094753912
Average loss at step  1256000 :  4.24816828835
Average loss at step  1258000 :  4.48902932966
Average loss at step  1260000 :  4.41821446526
Nearest to 音乐: 乐团, 演奏, 作曲家, 表演, 歌手, 歌曲, 唱片, 专辑,
Nearest to 地方: 地区, 市, 区域, 机关, 事务, 政府, 当地, 一带,
Nearest to 社会: 精神, 教育, 自我, 致力于, 政策, 个人, 法律, 文化,
Nearest to 服务: 免费, 邨, 来往, 往返, 网站, 总站, 路线, 屯门,
Nearest to 均: 皆, 都, 仍, 亦, 则, 宴请, 顾及, 一致,
Nearest to 型: 机车, 发动机, WDW, 装备, 辆, 车型, 车体, 动力,
Nearest to 学生: 学校, 课程, 教师, 该校, 中学, 小学, 学习, 教育,
Nearest to 今: 今属, 河南, 山西, 镇, 一带, 黄河, 二年, 民国,
Nearest to 受到: 受, 遭到, 引起, 深受, 对, 造成, 因素, 遭受,
Nearest to 事件: 事故, 行动, 报导, 行为, 革命

Nearest to 经济: 产业, 政策, 农业, 工资, 价值, 社会, 金融, 工业,
Average loss at step  1362000 :  3.73535951197
Average loss at step  1364000 :  3.2530463531
Average loss at step  1366000 :  3.86569560608
Average loss at step  1368000 :  3.61294563919
